# 1 - Importation des modules

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import joblib
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV


# 2 - Chargement et copie des données

In [2]:
data = pd.read_csv('./../Data_Arbre.csv')
df = data.copy()
print(df.head(5))

   longitude   latitude              clc_quartier             clc_secteur  \
0   3.293264  49.840500  Quartier du Centre-Ville             Quai Gayant   
1   3.273380  49.861409    Quartier du Vermandois              Stade Cepy   
2   3.289068  49.844513  Quartier du Centre-Ville   Rue Villebois Mareuil   
3   3.302387  49.861778      Quartier de l'Europe  Square des Marronniers   
4   3.304047  49.858446      Quartier de l'Europe           Avenue Buffon   

   haut_tot  haut_tronc  tronc_diam fk_arb_etat fk_stadedev     fk_port  \
0       6.0         2.0        37.0    EN PLACE       Jeune  semi libre   
1      13.0         1.0       160.0    EN PLACE      Adulte  semi libre   
2      12.0         3.0       116.0    REMPLACÉ      Adulte  semi libre   
3      16.0         3.0       150.0    EN PLACE      Adulte  semi libre   
4       5.0         2.0       170.0    Essouché      Adulte      réduit   

  fk_pied fk_situation fk_revetement  age_estim  fk_prec_estim  clc_nbr_diag  \
0   ga

# 3 - Selection et divison des données utiles et de la cible

In [3]:
# Sélectionner les colonnes pertinentes
selected_columns = ['longitude', 'latitude', 'haut_tot', 'haut_tronc', 'tronc_diam', 'age_estim',
                    'fk_stadedev', 'fk_port', 'fk_pied', 'fk_situation', 'fk_revetement', 'villeca',
                    'feuillage', 'remarquable']

# Séparation des colonnes numériques, catégorielles et booléennes
numerical = ['longitude', 'latitude', 'haut_tot', 'haut_tronc', 'tronc_diam', 'age_estim']
categorical = ['fk_stadedev', 'fk_port', 'fk_pied', 'fk_situation', 'villeca', 'feuillage']
categorical_bool = ['remarquable', 'fk_revetement']
target = 'fk_arb_etat'

# 4 - Creation des données de test et de train

In [4]:
# Définir X et y
X = df[selected_columns]
y = df[target]

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5 - Enregistrement du Json test

In [5]:

dataframe = pd.concat([X_test, y_test], axis=1)
dataframe.to_json('data_test.json', orient='records', lines=True)

# 6 - Encodage des données

In [6]:

y_train = y_train.apply(lambda x: 1 if x == 'Essouché' else 0)
y_test = y_test.apply(lambda x: 1 if x == 'Essouché' else 0)


# Standardiser les données numériques
scaler = StandardScaler()
scaler.fit(X[numerical])
X_train_numerical = pd.DataFrame(scaler.transform(X_train[numerical]), columns=numerical)
X_test_numerical = pd.DataFrame(scaler.transform(X_test[numerical]), columns=numerical)

# Encodage des données catégorielles
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoder.fit(X[categorical])
X_train_categorical = pd.DataFrame(encoder.transform(X_train[categorical]), columns=encoder.get_feature_names_out(categorical))
X_test_categorical = pd.DataFrame(encoder.transform(X_test[categorical]), columns=encoder.get_feature_names_out(categorical))

# Encodage des données booléennes
encoder_bool = OrdinalEncoder(categories=[['Non', 'Oui'] for _ in categorical_bool])
encoder_bool.fit(X[categorical_bool])
X_train_bool = pd.DataFrame(encoder_bool.transform(X_train[categorical_bool]), columns=categorical_bool)
X_test_bool = pd.DataFrame(encoder_bool.transform(X_test[categorical_bool]), columns=categorical_bool)

# Concaténation des données
X_train_encoded = pd.concat([X_train_numerical, X_train_categorical, X_train_bool], axis=1)
X_test_encoded = pd.concat([X_test_numerical, X_test_categorical, X_test_bool], axis=1)


# 6 - Equilibrage des classes avec SMOTE

In [7]:
smote = SMOTE(random_state=42, sampling_strategy=0.25)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_encoded, y_train)


# 7 - Entrainement model de selection et selection des 20 features les plus importante

In [8]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_resampled, y_train_resampled)

# Importance des caractéristiques
importance = clf.feature_importances_
feature_importance = pd.DataFrame(importance, index=X_train_encoded.columns, columns=['importance']).sort_values('importance', ascending=False)
top_20_features = feature_importance.head(20).index

# Utiliser les caractéristiques les plus importantes
X_train_subset = X_train_resampled[top_20_features]

# 8 - Instanciation et entrainement du model RandomForest Final

In [9]:
clf_subset = RandomForestClassifier(random_state=42)
clf_subset.fit(X_train_subset, y_train_resampled)
X_test_subset = X_test_encoded[top_20_features]

# 9 - Exemple prediction des resultat et Score / Matrice 


In [10]:

y_pred = clf_subset.predict(X_test_subset)
y_pred_proba = clf_subset.predict_proba(X_test_subset)[:, 1]


conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9831309041835358
Confusion Matrix:
 [[1448    9]
 [  16    9]]


# 10 - Sauvegarde du model et des encoders avec joblib

In [11]:

# Sauvegarder les objets avec joblib
model_artifacts = {
    'scaler': scaler,
    'encoder': encoder,
    'encoder_bool': encoder_bool,
    'random_forest_model': clf,
    'random_forest_model_subset': clf_subset,
    'top_20_features': top_20_features
}

joblib.dump(model_artifacts, 'model_artifacts.pkl')

print("Les objets ont été sauvegardés avec succès.")


Les objets ont été sauvegardés avec succès.
